In [1]:
# ========================================
# STEP 1: Imports, Env Load, and Tool Definition
# ========================================
import os
from dotenv import load_dotenv
from openai import AzureOpenAI

# Agent Framework Imports
# We are NOT using AzureCliCredential, as we are using the API Key
from agent_framework.azure import AzureOpenAIChatClient
from agent_framework import ChatAgent

print("Imports complete.")

# Load environment variables
load_dotenv()
endpoint_model = os.getenv("AZURE_MODEL_ENDPOINT")
deployment = os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME")
model_key = os.getenv("AZURE_AI_MODEL_API_KEY")

print("Environment variables loaded.")

# ========================================
# CUSTOM TOOL (The Project Core)
# ========================================

def calculate_employment_cost(number_of_people: int, contract_months: int, minimum_wage: float) -> str:
    """
    Calculates the total cost of a green employment program.
    The agent MUST have this information to use the tool.

    Args:
        number_of_people (int): The total number of people to be hired.
        contract_months (int): The number of months for the contract.
        minimum_wage (float): The wage value (e.g., 1518.00).
    
    Returns:
        str: A formatted string with the total cost message in BRL (R$).
    """
    
    # This is the "1 functional action"
    total_cost = number_of_people * contract_months * minimum_wage
    
    # Format the response in BRL
    # Using 'pt_BR.UTF-8' locale for correct currency formatting might be complex in Codespaces,
    # so we'll use a simple f-string format.
    result = f"The estimated cost to hire {number_of_people} people for {contract_months} months, at a wage of BRL {minimum_wage:,.2f}, is BRL {total_cost:,.2f}."
    
    print(f"[Tool Debug]: 'calculate_employment_cost' function was called. Result: {result}")
    return result

print("✅ Tool 'calculate_employment_cost' defined successfully!")

Imports complete.
Environment variables loaded.
✅ Tool 'calculate_employment_cost' defined successfully!


In [2]:
# ========================================
# STEP 2: Creating the "ESG Agent"
# ========================================

async def create_esg_agent():
    """
    Creates an ESG specialist agent with the cost calculator tool.
    """
    # Authentication is handled here by the API key
    chat_client = AzureOpenAIChatClient(
        endpoint=endpoint_model,
        api_key=model_key,
        deployment_name=deployment,
        api_version="2024-02-15-preview", # Using our validated API version
    )
    
    # ========================================
    # ESG AGENT (Based on the MBA Thesis) 🌳
    # ========================================
    esg_agent = ChatAgent(
        chat_client=chat_client,
        name="esg_cost_calculator_agent",
        
        # Agent's "Brain": Instructions in English
        instructions=(
            "You are an expert assistant for ESG and Public Policy, focused on data from Brazil. "
            "Your job is to help public managers estimate the costs of green employment programs. "
            "When the user asks 'how much does it cost' (or 'what is the cost', 'estimate', 'calculate'), "
            "you MUST use the 'calculate_employment_cost' tool. "
            "To do this, you need 3 values: number_of_people, contract_months, and minimum_wage. "
            "If the user does not provide all 3 values, politely ask for them. "
            "Always respond in English, but specify the currency as BRL (R$)."
        ),
        
        # Giving the "Tool" to the agent
        tools=[calculate_employment_cost] 
    )

    # Return only the agent
    return esg_agent

print("✅ Function 'create_esg_agent' defined successfully!")

✅ Function 'create_esg_agent' defined successfully!


In [3]:
# ========================================
# STEP 3: Running the ESG Agent! 🚀
# ========================================

# The question, now in English to match the agent's instructions
user_question = "Hello! How much would it cost to hire 1,500 people for 6 months, at a minimum wage of 1518.00 BRL?"

async def run_esg_agent(question_from_user):
    
    # 1. Create the agent
    esg_agent = await create_esg_agent()

    print(f"Sending question to ESG Agent: '{question_from_user}'")
    print("...awaiting AI response...")
    
    # 2. Run the agent
    result = await esg_agent.run(question_from_user)
    
    # 3. Show the final answer
    print("\n=== ESG Agent Response ===")
    print(result)
    
    print("\n--- Process Complete ---")

# =C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C=
# THIS IS THE MOMENT TO TAKE THE SCREENSHOT FOR YOUR ENGLISH README! 📸
# =C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C-C=
await run_esg_agent(user_question)

Sending question to ESG Agent: 'Hello! How much would it cost to hire 1,500 people for 6 months, at a minimum wage of 1518.00 BRL?'
...awaiting AI response...
[Tool Debug]: 'calculate_employment_cost' function was called. Result: The estimated cost to hire 1500 people for 6 months, at a wage of BRL 1,518.00, is BRL 13,662,000.00.

=== ESG Agent Response ===
The estimated cost to hire 1,500 people for 6 months, at a minimum wage of R$1,518.00, is R$13,662,000.00.

--- Process Complete ---
